In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
from keras.preprocessing import image
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [2]:
train_data_dir = r"C:\Users\kezin\Downloads\dataset2\train"
validation_data_dir = r"C:\Users\kezin\Downloads\dataset2\test"

Define image data generators with augmentation

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)


def augment_data(directory, target_count=3000):
    for class_label in os.listdir(directory):
        class_path = os.path.join(directory, class_label)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            num_files = len(images)
            
            # If the number of images is greater than 1000, randomly select 1000 images
            if num_files > target_count:
                print(f"Class {class_label} has {num_files} images. Randomly selecting {target_count} images.")
                images_to_keep = random.sample(images, target_count)
                for img_name in images:
                    if img_name not in images_to_keep:
                        os.remove(os.path.join(class_path, img_name))
            
            # If the number of images is less than 1000, augment the data
            elif num_files < target_count:
                print(f"Augmenting class {class_label} with {num_files} images to {target_count} images")
                datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    shear_range=0.3,
                    zoom_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest')
                
                existing_images = []
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
                    img = image.img_to_array(img)
                    img = np.expand_dims(img, axis=0)
                    existing_images.append(img)
                
                existing_images = np.vstack(existing_images)
                i = 0
                for batch in datagen.flow(existing_images, batch_size=1, save_to_dir=class_path, save_prefix='aug', save_format='jpg'):
                    i += 1
                    if i > target_count - num_files:
                        break

# Augment training data
augment_data(train_data_dir)


Class angry has 3995 images. Randomly selecting 3000 images.
Augmenting class disgust with 436 images to 3000 images
Class fear has 4097 images. Randomly selecting 3000 images.
Class happy has 7215 images. Randomly selecting 3000 images.
Class neutral has 4965 images. Randomly selecting 3000 images.
Class sad has 4830 images. Randomly selecting 3000 images.
Class surprise has 3171 images. Randomly selecting 3000 images.


Create generators

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

Found 21001 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

img, label = train_generator.__next__()

Define the model

In [6]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,095 (9.50 MB)

 Trainable params: 2,489,095 (9.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
# Calculate the number of training and testing images
num_train_imgs = sum([len(files) for r, d, files in os.walk(train_data_dir)])
num_test_imgs = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

print(num_train_imgs)
print(num_test_imgs)

21001
7178


In [8]:
epochs = 30
history = model.fit(
    train_generator,
    steps_per_epoch=num_train_imgs // 32,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_test_imgs // 32)

c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 377s 571ms/step - accuracy: 0.1451 - loss: 1.9479 - val_accuracy: 0.1159 - val_loss: 1.9454
Epoch 2/30
  1/656 ━━━━━━━━━━━━━━━━━━━━ 1:18 120ms/step - accuracy: 0.0312 - loss: 1.9504

c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


656/656 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0312 - loss: 1.9504 - val_accuracy: 0.1158 - val_loss: 1.9454
Epoch 3/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 105s 160ms/step - accuracy: 0.1371 - loss: 1.9462 - val_accuracy: 0.1719 - val_loss: 1.9442
Epoch 4/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0625 - loss: 1.9472 - val_accuracy: 0.1716 - val_loss: 1.9442
Epoch 5/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 118s 179ms/step - accuracy: 0.1433 - loss: 1.9460 - val_accuracy: 0.1159 - val_loss: 1.9462
Epoch 6/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.1562 - loss: 1.9459 - val_accuracy: 0.1155 - val_loss: 1.9461
Epoch 7/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 102s 155ms/step - accuracy: 0.1443 - loss: 1.9460 - val_accuracy: 0.1716 - val_loss: 1.9465
Epoch 8/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.1250 - loss: 1.9450 - val_accuracy: 0.1719 - val_loss: 1.9465
Epoch 9/30
656/656 ━━━━━━━━━━━━━━━━━━━━ 97s 148ms/step - accuracy: 0.1417 - loss: 1.9460 - val_ac

Save the model

In [9]:
model.save('model_file.h5')

reports

In [10]:
# Path to your test dataset
test_dir = r"C:\Users\kezin\Downloads\archive\test"  # Update this to your test image folder

# Image parameters (should match your model's input size)
IMG_SIZE = (48, 48)  # Update if different
BATCH_SIZE = 32  # Adjust batch size as needed

# Load test images using ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Ensures labels are one-hot encoded
    color_mode='grayscale',  # Force grayscale images (fix for input shape mismatch)
    shuffle=False  
)

# Get true labels
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())  # Get class names

# Predict on test data
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=class_labels)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)


Found 7178 images belonging to 7 classes.
225/225 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step
Accuracy: 0.1427
F1 Score: 0.0356
Confusion Matrix:
[[   0    0  958    0    0    0    0]
 [   0    0  111    0    0    0    0]
 [   0    0 1024    0    0    0    0]
 [   0    0 1774    0    0    0    0]
 [   0    0 1233    0    0    0    0]
 [   0    0 1247    0    0    0    0]
 [   0    0  831    0    0    0    0]]

Classification Report:
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       958
     disgust       0.00      0.00      0.00       111
        fear       0.14      1.00      0.25      1024
       happy       0.00      0.00      0.00      1774
     neutral       0.00      0.00      0.00      1233
         sad       0.00      0.00      0.00      1247
    surprise       0.00      0.00      0.00       831

    accuracy                           0.14      7178
   macro avg       0.02      0.14      0.04      7178
weighted avg       0.02      0

c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kezin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo